In [1]:
# importando o drive da biblioteca do google
# e montando o drive para poder ter acesso aos arquivos locais

from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
# importação das bibliotecas de manipulação de dados (pandas) e cálculos algébricos (numpy)

import pandas as pd
import numpy as np

In [0]:
# leitura dos dados através do método read_csv() da biblioteca pandas

train = pd.read_csv('/content/drive/My Drive/kaggle_competitions/titanic/train.csv')
test = pd.read_csv('/content/drive/My Drive/kaggle_competitions/titanic/test.csv')
gender_submission = pd.read_csv('/content/drive/My Drive/kaggle_competitions/titanic/gender_submission.csv')

**Dados de treino**

In [4]:
# analisando os dados de treino, o método head() permite a visualização das 5 primeiras instâncias do dataset

train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [5]:
# o método shape permite que visualizemos o formato da matriz, quantidades de linhas e colunas
# como podemos perceber nosso dataset tem 891 instâncias e tem dimensão 12

train.shape

(891, 12)

In [6]:
# através do método isna() podemos verificar quais dados são nulos e quais não são
# em composição com a função sum(), podemos ter um total de quantos instâncias são nulas em cada atributo

train.isna().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [0]:
# A biblioteca SimpleImpute nos permite preencher os dados faltantes pela 
# média (mean), moda (most_frenquent), mediana (median) ou um valor contante (constant).

from sklearn.impute import SimpleImputer

In [0]:
# aqui usamos a estratégia de média para substituir valores nulos

imputer = SimpleImputer(missing_values=np.nan, strategy='mean')

In [0]:
# aqui usamos a estratégia de moda para substituir valores nulos

imputer2 = SimpleImputer(missing_values=np.nan, strategy='most_frequent')

In [0]:
# aqui aplicamos então a primeira estratégia (média) para preencher os valores nulos da feature Age

train[['Age']] = imputer.fit_transform(train[['Age']])

In [0]:
# aqui aplicamos então a segunda estratégia (moda) para preencher os valores nulos da feature Embarked

train[['Embarked']] = imputer2.fit_transform(train[['Embarked']])

In [0]:
# decide remover duas features do dataset, o primeiro foi o Id do Passageiro, pois é irrelevante para a análise dos dados
# podendo até atrapalhar na análise. O segundo foi a Cabine do passageiro, pois haviam muitos dados faltantes e nenhuma estratégia
# de substituição dos valores faltantes daria certo.

train.drop(columns=['PassengerId', 'Cabin'], axis=1, inplace=True)

**Dados de teste**

In [15]:
# verificando os valores do dataset pelo método head()

test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S


In [17]:
# proporções do conjunto de dados
# 418 instâncias e dimensão 11

test.shape

(418, 11)

In [18]:
test.isna().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [0]:
from sklearn.impute import SimpleImputer

In [0]:
imputer3 = SimpleImputer(missing_values=np.nan, strategy='mean')

In [0]:
test[['Age', 'Fare']] = imputer3.fit_transform(test[['Age', 'Fare']])

In [0]:
submission = pd.DataFrame()

In [0]:
submission['PassengerId'] = test['PassengerId']

In [0]:
test.drop(columns=['PassengerId', 'Cabin'], axis=1, inplace=True)

**Exemplo para envio**

In [25]:
gender_submission.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [26]:
gender_submission.shape

(418, 2)

**Pré-Processamento**

In [0]:
sobreviveu = train['Survived']

In [0]:
train.drop(columns=['Survived'], axis=1, inplace=True)

In [0]:
from sklearn.preprocessing import LabelEncoder

In [0]:
encoder = LabelEncoder()

In [31]:
encoder.fit(np.unique(train['Embarked'].unique().tolist() + test['Embarked'].unique().tolist()))

LabelEncoder()

In [0]:
train['Embarked'] = encoder.transform(train['Embarked'])

In [0]:
test['Embarked'] = encoder.transform(test['Embarked'])

In [0]:
for useCol in train.columns.tolist()[:-1]:
  le = LabelEncoder().fit(np.unique(train[useCol].unique().tolist()+test[useCol].unique().tolist()))
  train[useCol] = le.transform(train[useCol])
  test[useCol] = le.transform(test[useCol])

In [0]:
train['Survived'] = sobreviveu

In [38]:
train.shape, test.shape

((891, 10), (418, 9))

In [0]:
X = train.iloc[:, :-1]
y = train.iloc[:, -1]

In [0]:
from sklearn.model_selection import train_test_split

In [0]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, stratify=y, random_state=42)

In [42]:
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((712, 9), (179, 9), (712,), (179,))

In [0]:
from sklearn.preprocessing import StandardScaler

In [0]:
scale = StandardScaler()
X_train = scale.fit_transform(X_train)
X_test = scale.transform(X_test)

In [45]:
import keras
from keras.models import Sequential
from keras.layers import Dense

Using TensorFlow backend.


In [46]:
model = Sequential()

In [47]:
model.add(Dense(units=5, activation='relu', input_dim=9))

In [0]:
model.add(Dense(units=5, activation='relu'))

In [0]:
model.add(Dense(units=1, activation='sigmoid'))

In [50]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])



Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [51]:
model.fit(X_train, y_train, batch_size=10, epochs=100)




Epoch 1/100





712/712 [==============================] - 1s 1ms/step - loss: 0.7212 - acc: 0.4789
Epoch 2/100
712/712 [==============================] - 0s 118us/step - loss: 0.6736 - acc: 0.6419
Epoch 3/100
712/712 [==============================] - 0s 134us/step - loss: 0.6544 - acc: 0.6643
Epoch 4/100
712/712 [==============================] - 0s 139us/step - loss: 0.6390 - acc: 0.6798
Epoch 5/100
712/712 [==============================] - 0s 129us/step - loss: 0.6218 - acc: 0.6924
Epoch 6/100
712/712 [==============================] - 0s 132us/step - loss: 0.5996 - acc: 0.7051
Epoch 7/100
712/712 [==============================] - 0s 141us/step - loss: 0.5737 - acc: 0.7374
Epoch 8/100
712/712 [==============================] - 0s 144us/step - loss: 0.5461 - acc: 0.7584
Epoch 9/100
712/712 [==============================] - 0s 147us/step - loss: 0.5207 - acc: 0.7626
Epoch 10/100
712/712 [==============================] - 0s 147us/step - loss: 0.4976 - acc: 0.7753
Epoch 11/100


In [0]:
y_pred = model.predict(X_test)

In [0]:
y_pred = (y_pred > 0.5)

In [0]:
from sklearn.metrics import accuracy_score, confusion_matrix

In [55]:
accuracy_score(y_test, y_pred)

0.7932960893854749

In [56]:
confusion_matrix(y_test, y_pred)

array([[98, 12],
       [25, 44]])

In [0]:
std = StandardScaler()
X = std.fit_transform(X)
test = std.transform(test)

In [0]:
classificador = Sequential()

In [0]:
classificador.add(Dense(units=5, activation='relu', input_dim=9))

In [0]:
classificador.add(Dense(units=9, activation='relu'))

In [0]:
classificador.add(Dense(units=1, activation='sigmoid'))

In [0]:
classificador.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [63]:
classificador.fit(X, y, batch_size=10, epochs=100)

Epoch 1/100
891/891 [==============================] - 0s 451us/step - loss: 0.6112 - acc: 0.6330
Epoch 2/100
891/891 [==============================] - 0s 149us/step - loss: 0.5688 - acc: 0.7082
Epoch 3/100
891/891 [==============================] - 0s 128us/step - loss: 0.5449 - acc: 0.7486
Epoch 4/100
891/891 [==============================] - 0s 150us/step - loss: 0.5257 - acc: 0.7744
Epoch 5/100
891/891 [==============================] - 0s 132us/step - loss: 0.5111 - acc: 0.7834
Epoch 6/100
891/891 [==============================] - 0s 138us/step - loss: 0.4960 - acc: 0.7957
Epoch 7/100
891/891 [==============================] - 0s 135us/step - loss: 0.4846 - acc: 0.7935
Epoch 8/100
891/891 [==============================] - 0s 122us/step - loss: 0.4751 - acc: 0.7924
Epoch 9/100
891/891 [==============================] - 0s 131us/step - loss: 0.4680 - acc: 0.8013
Epoch 10/100
891/891 [==============================] - 0s 130us/step - loss: 0.4615 - acc: 0.8081
Epoch 11/100
891/89

In [0]:
prediction = classificador.predict(test)

In [0]:
prediction = (prediction > 0.5)

In [0]:
submission['Survived'] = prediction

In [0]:
submission['Survived'] = np.where(submission['Survived'] == False, 0, 1)

In [0]:
submission.to_csv('/content/drive/My Drive/kaggle_competitions/titanic/submission.csv', index=False)